# Projet python

In [1]:
import pandas as pd
from Scrapper import Scrapper
from Tools import generate_date_ranges_months

Nous traitons avec des données d'incendies en France.  
On cherche à savoir si ces évènements sont concentrés dans des endroits précis en France, ou bien ces évènements sont purement aléatoires.

## Scraping

Tout d'abord, définissons la période auquel nous voulons analysés nos données.  
Nous choisissons de prendre toute l'année 2023.  
  
Toutefois, nous ne pouvons pas juste récupérer l'année entière directement car il y a des problèmes de serveur avec le site où nous scrappons nos données.  
Nous séparons donc notre récupérations en chunk de 3 mois.

In [2]:
start_date = "01/01/2023"
end_date = "31/12/2023"
date_ranges = generate_date_ranges_months(start_date, end_date, months=2)
date_ranges

[('01/01/2023', '28/02/2023'),
 ('01/03/2023', '30/04/2023'),
 ('01/05/2023', '30/06/2023'),
 ('01/07/2023', '31/08/2023'),
 ('01/09/2023', '31/10/2023'),
 ('01/11/2023', '31/12/2023')]

In [3]:
dfs = []
for d in date_ranges:
  dfs.append(Scrapper().generate_dataframe(d[0], d[1]))
fire_departure_df = pd.concat(dfs, ignore_index=True)

12 max workers...
[('01/01/2023', '31/01/2023'), ('01/02/2023', '28/02/2023')]
Scrapping done successfully...
Scrapping done successfully...
12 max workers...
[('01/03/2023', '31/03/2023'), ('01/04/2023', '30/04/2023')]
Scrapping done successfully...
Scrapping done successfully...
12 max workers...
[('01/05/2023', '31/05/2023'), ('01/06/2023', '30/06/2023')]
Scrapping done successfully...
Scrapping done successfully...
12 max workers...
[('01/07/2023', '31/07/2023'), ('01/08/2023', '31/08/2023')]
Scrapping done successfully...
Scrapping done successfully...
12 max workers...
[('01/09/2023', '01/10/2023'), ('02/10/2023', '31/10/2023')]
Scrapping done successfully...
Scrapping done successfully...
12 max workers...
[('01/11/2023', '01/12/2023'), ('02/12/2023', '31/12/2023')]
Scrapping done successfully...
Scrapping done successfully...


In [4]:
fire_departure_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2694 entries, 0 to 2693
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0                           2694 non-null   object
 1   Année                   2694 non-null   object
 2   Alerte                  2694 non-null   object
 3   Département             2694 non-null   object
 4   Commune                 2694 non-null   object
 5   Surface (ha)            2694 non-null   object
 6   Nature                  2694 non-null   object
 7   Précision de la donnée  2694 non-null   object
dtypes: object(8)
memory usage: 168.5+ KB


Sauvegardons ces données brut scrapés dans un csv afin d'éviter de devoir toujours refaire du scraping.  
Nous les sauvegarderons dans un fichier nommé "raw_fire_departure_2023.csv".

In [5]:
fire_departure_df.to_csv("raw_fire_departure_2023.csv", index=False)

## Import data

Pour éviter de refaire le scraping prédéfini en haut, nous importons le fichier CSV qu'on a enregistré au préalable.

In [11]:
fire_departure_df = pd.read_csv("raw_fire_departure_2023.csv")
fire_departure_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2694 entries, 0 to 2693
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              0 non-null      float64
 1   Année                   2694 non-null   int64  
 2   Alerte                  2694 non-null   object 
 3   Département             2694 non-null   object 
 4   Commune                 2694 non-null   object 
 5   Surface (ha)            2694 non-null   float64
 6   Nature                  2694 non-null   object 
 7   Précision de la donnée  2694 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 168.5+ KB


## Data cleaning

Commençons par supprimer les données qui sont doubles.

In [6]:
doublons = fire_departure_df[fire_departure_df.duplicated()]
print(f"Il y a {len(doublons)} données en doubles...")

Il y a 25 données en doubles...


In [7]:
fire_departure_df.drop_duplicates()
fire_departure_df.reset_index(inplace=True)
fire_departure_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2694 entries, 0 to 2693
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   index                   2694 non-null   int64 
 1                           2694 non-null   object
 2   Année                   2694 non-null   object
 3   Alerte                  2694 non-null   object
 4   Département             2694 non-null   object
 5   Commune                 2694 non-null   object
 6   Surface (ha)            2694 non-null   object
 7   Nature                  2694 non-null   object
 8   Précision de la donnée  2694 non-null   object
dtypes: int64(1), object(8)
memory usage: 189.6+ KB


Gardons uniquement les colonnes qui nous intéressent.

In [9]:
fire_departure_df.columns

Index(['index', '', 'Année', 'Alerte', 'Département', 'Commune',
       'Surface (ha)', 'Nature', 'Précision de la donnée'],
      dtype='object')

In [10]:
res = fire_departure_df.drop(columns=["index", "Précision de la donnée"], axis=1)
res.head()

,,Année,Alerte,Département,Commune,Surface (ha),Nature
0,,2023,01/01/2023 10:05,2B,Cagnano,0.0100,Malveillance
1,,2023,01/01/2023 15:29,88,Dommartin-lès-Remiremont,0.0002,-
2,,2023,01/01/2023 17:54,40,Labatut,0.1000,Accidentelle
3,,2023,01/01/2023 18:11,64,Iholdy,1.0000,-
4,,2023,02/01/2023 20:59,83,Rayol-Canadel-sur-Mer,0.0035,Involontaire (particulier)


## Vizualisation

In [ ]:
import matplotlib.pyplot as plt